In [10]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from datetime import datetime
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict
from transformers import BertTokenizer,BertModel,BertTokenizerFast,DebertaTokenizer,DebertaTokenizerFast, AutoTokenizer
from utils.MIND import MIND
from utils.Manager import Manager

logger = logging.getLogger(__name__)

In [11]:
t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = DebertaTokenizerFast.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")

In [2]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'

# config.signal_length = 10
# config.scale = 'large'
# config.impr_size = 100
# config.mode = 'dev'
# config.scale = 'large'

# config.bert = 'microsoft/deberta-base'
# config.embedding = 'deberta'

# config.bert = 'bert-base-uncased
# config.embedding = 'bert'

manager = Manager(config)
# manager.gather_same_user_impr()
# a = MIND(manager)

In [3]:
from utils.MIND import MIND_news, MIND_history
b = MIND_news(manager)
c = MIND_history(manager)

[2021-10-04 15:40:40,253] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDdemo_train/news.pkl
[2021-10-04 15:40:41,033] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/bert/MINDdemo_train/10/behaviors.pkl
[2021-10-04 15:40:41,046] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDdemo_train/news.pkl
[2021-10-04 15:40:41,837] INFO (utils.utils) deduplicating...


In [8]:
c[1]

{'user_id': 2,
 'his_id': array([29991,  1537, 28125,  6875, 20529, 25906, 26638,  1082, 13995,
        13919, 27868, 10693, 22638, 25937,  5642, 19188,   129, 10970,
         4853, 22037, 26144,  2600,  4501,  2757,  8178, 14857, 30635,
        17680, 28583, 28646, 17080, 29418, 28192, 25725, 15871, 17164,
        23107, 28920,  2394, 28044, 21717, 22954, 26998, 12733, 14202,
        13463, 12087,  8524,  2334, 25816]),
 'his_encoded_index': array([[  101,  3053,  1002, ...,     0,     0,     0],
        [  101,  3674,  3506, ...,  7732,  4534,  1012],
        [  101,  5615,  5338, ...,     0,     0,     0],
        ...,
        [  101, 22896,  8909, ...,  2027,  4995,  2102],
        [  101,  3945, 13746, ...,     0,     0,     0],
        [  101, 10427,  8787, ...,     0,     0,     0]]),
 'his_attn_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0],
  

In [ ]:
a = c

news = t.convert_ids_to_tokens(a[0]['his_encoded_index'][1])
mask = a[0]['his_attn_mask'][1]
word = manager.convert_tokens_to_words(news)
subword = a[0]['his_subword_index'][1]
dedup = a[0]['his_refined_mask'][1]

for i,j,k,z in zip(news,mask,subword,dedup):
    print(i,j,k,z)

In [12]:
index = 2

In [13]:
a = [1,2,3]
a[index]

IndexError: list index out of range

In [ ]:
x = a[1]
x['user_index']

In [ ]:
cdd = x['cdd_encoded_index'][0]

In [ ]:
subword = x['cdd_subword_index'][0]

In [ ]:
manager.convert_tokens_to_words(t.convert_ids_to_tokens(cdd))

In [ ]:
subword

In [ ]:
t.convert_ids_to_tokens(cdd)

In [ ]:
t.convert_ids_to_tokens([   1, 7565,   16, 4690,    0,    0,    0,    0])

In [ ]:
tokens = t.tokenize("[CLS] I love you embeddings.")
tokens

In [ ]:
news = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/deberta/MINDdemo_train/news.pkl', 'rb'))
ids = news['encoded_news']
subwords = news['subwords_all']

In [ ]:
t.convert_tokens_to_ids([i for i in r"[.&*()+=/\<>,!?;:~`@#$%^]"])